In [1]:
########################################
import os
import re
import json
import random
import pickle
import operator
import itertools

import numpy
import sklearn.ensemble
import sklearn.linear_model
import sklearn.feature_extraction

#import raha
########################################


In [ ]:
def value_normalizer(value):
        """
        This method takes a value and minimally normalizes it.
        """
        value = html.unescape(value)
        value = re.sub("[\t\n ]+", " ", value, re.UNICODE)
        value = value.strip("\t\n ")
        return value

In [ ]:
def read_csv_dataset(dataset_path):
        """
        This method reads a dataset from a csv file path.
        """
        dataframe = pandas.read_csv(dataset_path, sep=",", header="infer", encoding="utf-8", dtype=str,
                                    keep_default_na=False, low_memory=False).applymap(value_normalizer)
        return dataframe


In [ ]:
name = dataset_dictionary["name"]
path = dataset_dictionary["path"]
dataframe = self.read_csv_dataset(dataset_dictionary["path"])
if "clean_path" in dataset_dictionary:
    has_ground_truth = True
    clean_path = dataset_dictionary["clean_path"]
    clean_dataframe = read_csv_dataset(dataset_dictionary["clean_path"])
if "repaired_path" in dataset_dictionary:
    has_been_repaired = True
    repaired_path = dataset_dictionary["repaired_path"]
    repaired_dataframe = read_csv_dataset(dataset_dictionary["repaired_path"])


In [2]:
DATASET_CONSTRAINTS = {
            "hospital": {
                "functions": [["city", "zip"], ["city", "county"], ["zip", "city"], ["zip", "state"], ["zip", "county"],
                              ["county", "state"]],
                "patterns": [["index", "^[\d]+$", "ONM"], ["provider_number", "^[\d]+$", "ONM"],
                             ["zip", "^[\d]{5}$", "ONM"], ["state", "^[a-z]{2}$", "ONM"], ["phone", "^[\d]+$", "ONM"]]
            },
            "flights": {
                "functions": [["flight", "act_dep_time"], ["flight", "sched_arr_time"], ["flight", "act_arr_time"],
                              ["flight", "sched_dep_time"]],
                "patterns": []
            },
}

In [2]:
dataset_name = "flights"
dataset_dictionary = {
        "name": dataset_name,
        "path": os.path.abspath(os.path.join(os.path.dirname(__file__), os.pardir, "datasets", dataset_name, "dirty.csv")),
        "clean_path": os.path.abspath(os.path.join(os.path.dirname(__file__), os.pardir, "datasets", dataset_name, "clean.csv"))
    }

NameError: name 'os' is not defined

In [ ]:
def run_dboost(dd):
        """
        This method runs dBoost.
        """
        if self.VERBOSE:
            print("------------------------------------------------------------------------\n"
                  "------------------------------Running dBoost----------------------------\n"
                  "------------------------------------------------------------------------")
        d = raha.dataset.Dataset(dd)
        sp_folder_path = os.path.join(os.path.dirname(dd["path"]), "raha-baran-results-" + d.name, "strategy-profiling")
        strategy_profiles_list = [pickle.load(open(os.path.join(sp_folder_path, strategy_file), "rb"))
                                  for strategy_file in os.listdir(sp_folder_path)]
        random_tuples_list = [i for i in random.sample(range(d.dataframe.shape[0]), d.dataframe.shape[0])]
        labeled_tuples = {i: 1 for i in random_tuples_list[:int(d.dataframe.shape[0] / 100.0)]}
        best_f1 = -1.0
        best_strategy = ""
        detection_dictionary = {}
        for strategy_profile in strategy_profiles_list:
            algorithm = json.loads(strategy_profile["name"])[0]
            if algorithm == "OD":
                strategy_output = {cell: "JUST A DUUMY VALUE" for cell in strategy_profile["output"]}
                er = d.get_data_cleaning_evaluation(strategy_output, sampled_rows_dictionary=labeled_tuples)[:3]
                if er[2] > best_f1:
                    best_f1 = er[2]
                    best_strategy = strategy_profile["name"]
                    detection_dictionary = dict(strategy_output)
        return detection_dictionary